# Instantiate `DietVision` class

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

from diet_vision import DietVision
dv_instance = DietVision()

In [3]:
dv_instance.upload_image('fish_chips.jpg')

0 = [0]
1 = [1]
2 = [2]
3 = [3]
4 = [4]
5 = [5]
6 = [6]
7 = [7]
8 = [8]
9 = [9]
10 = [10]
11 = [11]
12 = [12]
13 = [13]
14 = [14]
15 = [15]
16 = [16]
17 = [17]
18 = [18]
19 = [19]
20 = [20]
21 = [21]
22 = [22]
23 = [23]
24 = [24]
25 = [25]
26 = [26]
27 = [27]
28 = [28]
29 = [29]
30 = [30]
31 = [31]
32 = [32]
33 = [33]
34 = [34]
35 = [35]
36 = [36]
37 = [37]
38 = [38]
39 = [39]
40 = [40]
41 = [41]
42 = [42]
43 = [43]
44 = [44]
45 = [45]
46 = [46]
47 = [47]
48 = [48]
49 = [49]
50 = [50]
51 = [51]
52 = [52]
53 = [53]
54 = [54]
55 = [55]
56 = [56]
57 = [57]
58 = [58]
59 = [59]
60 = [60]
61 = [61]
62 = [62]
63 = [63]
64 = [64]
65 = [65]
66 = [66]
67 = [67]
68 = [68]
69 = [69]
70 = [70]
71 = [71]
72 = [72]
73 = [73]
74 = [74]
75 = [75]
76 = [76]
77 = [77]
78 = [78]
79 = [79]
80 = [80]
81 = [81]
82 = [82]
83 = [83]
84 = [84]
85 = [85]
86 = [86]


'/Users/euijaekim/Projects/nyu_diet_vision/static/images/overlays/annotator_image_1691440728.png'

In [4]:
import numpy as np
np.save('./sam_result.npy', dv_instance.sam_result)

In [5]:
npy = np.load('./sam_result.npy', allow_pickle=True)
npy[0].keys()

dict_keys(['segmentation', 'area', 'bbox', 'predicted_iou', 'point_coords', 'stability_score', 'crop_box'])

In [7]:
dv_instance.get_all_food_data()

[]

In [ ]:
dv_instance.find_mask_index_list([1170, 991, 1666, 1073])
# [0, 8, 32, 42, 45, 69, 72, 74]

In [ ]:
for idx, dvd in enumerate(dv_instance.diet_vision_dictionary):
    data = Image.fromarray(dvd['mask'])
    data.save(f'mask_1_fish_chips_{idx}.png')

In [ ]:
import requests
import json

items = range(0, 42)
response = []
for item in items:
    url = f'https://api.nal.usda.gov/fdc/v1/foods/list?dataType=Foundation,SR%20Legacy&pageSize=200&pageNumber={item}&api_key=ljOu2YIUetK44MUIP1iW7PJzRGo7jdSTTJyzHA0k'
    res1 = requests.get(url)
    res2 = json.loads(res1.text) # list length 200
    
    for food in res2:
        response.append(food['description'])

food_dictionary = { 'foods': response }
len(food_dictionary['foods'])

# Serializing json
json_object = json.dumps(food_dictionary, indent=4)
 
# Writing to sample.json
with open('../static/js/usdaFoods.js', "w") as outfile:
    outfile.write(json_object)

In [ ]:
combined_mask2 = dv_instance.diet_vision_dictionary[0]['mask'].copy()

for idx in range(1, len(dv_instance.diet_vision_dictionary)):
    curr_mask = dv_instance.diet_vision_dictionary[idx]['mask'].copy()
    combined_mask2 = np.logical_or(combined_mask2, curr_mask)

In [ ]:
data = Image.fromarray(combined_mask2)
data.save('combined_mask2.png')

In [ ]:
combined_mask3 = dv_instance.raw_diet_vision_dictionary[0]['mask'].copy()

for idx in range(1, len(dv_instance.raw_diet_vision_dictionary)):
    curr_mask = dv_instance.raw_diet_vision_dictionary[idx]['mask'].copy()
    combined_mask3 = np.logical_or(combined_mask3, curr_mask)

data = Image.fromarray(combined_mask3)
data.save('combined_mask3.png')

In [ ]:
for idx in range(1, len(dv_instance.raw_diet_vision_dictionary)):
    nonzero_at = dv_instance.raw_diet_vision_dictionary[idx]['nonzero_at']

    if (1355, 967) in nonzero_at:
        print(f'idx = {idx}')

In [ ]:
import numpy as np

raw_diet_vision_dictionary = [{
    'mask': mask['segmentation'],
    'area': mask['area'],
    'class': None,
    'attached_to': idx,
    'nonzero_at': set(np.rec.fromarrays(np.nonzero(mask['segmentation'])).tolist())
} for idx, mask in enumerate(sorted(dv_instance.sam_result, key=lambda x: x['area'], reverse=False))]

height, width = raw_diet_vision_dictionary[0]['mask'].shape

mask_dictionary = np.full((height, width), -1)

for idx, curr_dvd in enumerate(raw_diet_vision_dictionary):
    for pixel in curr_dvd['nonzero_at']:
        mask_dictionary[pixel] = idx

        # curr_dvd['attached_to'] = self.mask_dictionary[pixel]

In [ ]:
import itertools as it
import numpy as np

masks = []
for idx in range(0, len(raw_diet_vision_dictionary)):
    mask = np.full((height, width), False)

    tups = np.where(mask_dictionary == idx)

    if tups[0].size > 0 and tups[0].size == tups[1].size:
        for row, col in it.zip_longest(tups[0], tups[1]):
            mask[row, col] = True

        masks.append(mask)

In [ ]:
from PIL import Image

for idx in range(0, len(masks)):
    data = Image.fromarray(masks[idx])
    data.save(f'4_my_mask_{idx}.png')

combined_mask = masks[0]

for item in masks[1:]:
    curr_mask = item.copy()
    combined_mask = np.logical_or(combined_mask, curr_mask)

data = Image.fromarray(combined_mask)
data.save('4_combined_mask.png')

In [ ]:
data = Image.fromarray(np.logical_or(masks[58], masks[61]))
data.save('test.png')

In [ ]:
import glob
import os

if not os.path.exists('../static/images/overlays/'):
    os.mkdir('../static/images/overlays/')
else :
    for file in glob.glob('../static/images/overlays/**.txt'):
        os.remove(file)

In [ ]:
aset = set([])
aset.add((1, 1))
aset.add((2, 2))
tuple(zip(*aset))

os.path.join('../app/', 'b', 'c')